<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/finetune-gpt-oss-20b/notebooks/3-finetune_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run CUREBench eval
- run prediction
- run eval

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [12]:
import os
import sys
import os.path
import glob

In [3]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone -b finetune-gpt-oss-20b https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 263 (delta 104), reused 92 (delta 50), pack-reused 88 (from 2)
Receiving objects: 100% (263/263), 3.03 MiB | 10.82 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [5]:
%cd CUREBench

/content/CUREBench


In [6]:
!git pull

Already up to date.


In [7]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 40.0 MB/s eta 0:00:00


In [8]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 253ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [9]:
!./env/bin/pip list

Package Version
------- -------
pip     25.3


In [10]:
!./env/bin/pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 5.2 MB/s  0:00:03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 29.2 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 102.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 108.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 155.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 56.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 32.6 MB/s  0:00:12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 158.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [13]:
# get sit packages path and add to sys.path
lib_path = glob.glob('./env/lib/python*')[0]
env_path = os.path.join(lib_path, 'site-packages')
if env_path not in sys.path:
    sys.path.append(env_path)
    print(f"Successfully added {env_path} to sys.path.")
else:
    print(f"{env_path} already in sys.path.")

Successfully added ./env/lib/python3.12/site-packages to sys.path.


### load dataset

In [14]:
from datasets import load_dataset

ds = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 19704
    })
})

In [16]:
train_data = ds['train']

In [17]:
train_data[0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

**system identity**

In [28]:
import json
from core.eval_framework import system_identity
print(system_identity)


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.


**convert single data example to harmony format**

In [25]:
def convert_to_harmony_format(data_item, system_prompt):
    """
    Convert a single data item to OpenAI Harmony format.

    Args:
        data_item: Dictionary with 'Question', 'Complex_CoT', and 'Response' keys
        system_prompt: System identity string

    Returns:
        Dictionary with 'messages' key containing the conversation
    """
    messages = [
        {'role': 'system',
            'content': system_prompt,
            'thinking': None},
        {'role': 'user',
            'content': data_item['Question'],
            'thinking': None},
        {'role': 'assistant',
            'content': data_item['Response'],
            'thinking': data_item['Complex_CoT']}
    ]
    return {'messages': messages}

In [31]:
converted_example = convert_to_harmony_format(train_data[0], system_identity)
print(json.dumps(converted_example, indent=2))

{
  "messages": [
    {
      "role": "system",
      "content": "\nYou are an expert medical assistant specializing in answering questions.\n\n**Your communication MUST strictly adhere to the Harmony channels:**\n1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.\n2.  **final:** Use this channel for the final output intended for the user.\n\n**Output Rule is Conditional:**\n* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.\n    * **Format:** `{\"answer\": \"<LETTER>\"}` (e.g., `{\"answer\": \"A\"}`)\n* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.\n\n**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.",
      "thinking": null
    },
    {
      "role

**convert training data**

In [34]:
formatted_training_data = [convert_to_harmony_format(item, system_identity) for item in train_data]

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [39]:
formatted_training_data[0]

{'messages': [{'role': 'system',
   'content': '\nYou are an expert medical assistant specializing in answering questions.\n\n**Your communication MUST strictly adhere to the Harmony channels:**\n1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.\n2.  **final:** Use this channel for the final output intended for the user.\n\n**Output Rule is Conditional:**\n* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.\n    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)\n* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.\n\n**Instruction:** Generate a complete response. The final output must use the \'final\' channel and adhere to the conditional format rule.',
   'thinking': None},
  {'role': 'user',
   'content': 'Given the sympt

In [40]:
conversation = tokenizer.apply_chat_template(formatted_training_data[0]['messages'], tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-10-31

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coh

### finetune setup

In [41]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.98 GiB. GPU 0 has a total capacity of 22.16 GiB of which 1.33 GiB is free. Process 3433 has 20.82 GiB memory in use. Of the allocated memory 20.43 GiB is allocated by PyTorch, and 188.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### load curebench val dataset

In [ ]:
import pandas as pd
from typing import Dict, List
from torch.utils.data import DataLoader
from dataset_utils import build_dataset
from core.eval_framework import GPTOSS20BModel
from openai_harmony import load_harmony_encoding

In [ ]:
def load_dataset_by_config(config_path):
  # load config file to get dataset info
  config = json.load(open(config_path, 'r')) if config_path else {}
  if 'dataset' in config:
      dataset_config = config['dataset']
      dataset_name = dataset_config.get('dataset_name', 'treatment')
  print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
  dataset_path = dataset_config.get("dataset_path")

  # build dataset
  dataset = build_dataset(
        dataset_config.get("dataset_path"),
    )
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
  dataset_list = []

  for batch in dataloader:
      question_type = batch[0][0]

      if question_type == "multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
      elif question_type == "open_ended_multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
              "meta_question": batch[4][0],
          })
      elif question_type == "open_ended":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
  return dataset_list

In [ ]:
val_data_config_path= "metadata_config_val.json"
val_data_list = load_dataset_by_config(val_data_config_path)


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


In [ ]:
dataset = val_data_list[:10]

In [ ]:
dataset

[{'question_type': 'multi_choice',
  'id': 'U9PHZ83RKYV8',
  'question': 'Which drug brand name is associated with the treatment of acne?\nA: Salicylic Acid\nB: Minoxidil\nC: Ketoconazole\nD: Fluocinonide',
  'answer': 'A'},
 {'question_type': 'open_ended_multi_choice',
  'id': 'vIGwm8qguXYi',
  'question': 'What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?',
  'answer': 'B',
  'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?\nA: Wait for the symptoms to resolve on their own.\nB: Inform their healthcare provider immediately and seek emergency medical care.\nC: Stop chem

In [ ]:
example = dataset[3]
print(example)

{'question_type': 'open_ended_multi_choice', 'id': 'WfWiWK0yULaX', 'question': 'Which of the following conditions is a contraindication for the use of Gadavist?', 'answer': 'B', 'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: Which of the following conditions is a contraindication for the use of Gadavist?\nA: Mild hypersensitivity reactions to Gadavist\nB: History of severe hypersensitivity reactions to Gadavist\nC: Renal impairment\nD: Liver dysfunction\n\n"}


In [ ]:
question = example["question"]
question_type = example["question_type"]

In [ ]:
# Format prompt
if question_type == "multi_choice":
    prompt = f"The following is a multiple choice question about medicine. Answer with a valid json containing the letter corresponding to the correct answer.\n\nQuestion: {question}\n\nAnswer:"
elif question_type == "open_ended_multi_choice" or question_type == "open_ended":
    prompt = f"The following is an open-ended question about medicine. Provide a comprehensive answer.\n\nQuestion: {question}\n\nAnswer:"

In [ ]:
print(f"prompt: {prompt}")

prompt: The following is an open-ended question about medicine. Provide a comprehensive answer.

Question: Which of the following conditions is a contraindication for the use of Gadavist?

Answer:


In [ ]:
system_identity = """
You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule."""

In [ ]:
# model = kit.model
model = GPTOSS20BModel("openai/gpt-oss-20b", reasoning_lvl="low", quantization="auto",
                       system_identity=system_identity)
model.load()

In [ ]:
# Pass this list to the stop_tokens argument
stop_sequences = [
    # 1. Stops the loop of internal reasoning tags
    "assistantfinal",

    # 2. Stops the common repetitive noise (often necessary)
    "analysisdone."
]

In [ ]:
response, reasoning_trace = model.inference(prompt,
    stop_strings=stop_sequences )

In [ ]:
response

'**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindication for Gadavist | Typical guidelines |\n|-----------|------------------------------------------|--------------------|\n| **Severe chronic kidney disease (CKD Stage 4–5)** – eGFR < 30\u202fmL/min/1.73\u202fm² or requiring dialysis | Reduced elimination of gadobutrol → higher risk of NSF and gadolinium‑related toxicity | Avoid unless essential; if needed, use the lowest effective dose and consider alternative imaging modalities |\n| **Acute kidney inju

In [ ]:
reasoning_trace

[{'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': 'Answer is open-ended? They phrased "Which of the following conditions is a contraindication for the use of Gadavist?" But no options given. Likely expecting mention kidney disease (renal impairment). So provide narrative.'}],
  'channel': 'analysis'},
 {'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': '**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindicatio

In [ ]:
print(model.system_identity)


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.


### write submission to file

In [ ]:
# Generate submission with metadata from config/args
print("Generating submission with metadata...")
submission_path = kit.save_submission_with_metadata(
    results=[results],
    filename="submission.csv",
    config_path=config_path,
    args=""
)

print(f"\n✅ Evaluation completed successfully!")
print(f"📊 Accuracy: {results.accuracy:.2%} ({results.correct_predictions}/{results.total_examples})")
print(f"📄 Submission saved to: {submission_path}")

# # Show metadata summary if verbose
# final_metadata = kit.get_metadata(getattr(args, 'config', None), args)
# print("\n📋 Final metadata:")
# for key, value in final_metadata.items():
#     print(f"  {key}: {value}")


### stopping criteria

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

In [ ]:
stopping_criteria = StoppingCriteriaList()

In [ ]:
StoppingCriteria

transformers.generation.stopping_criteria.StoppingCriteria

In [ ]:
stopping_criteria

[]

In [ ]:
kit.model.enc